In [1]:
import pandas as pd

In [2]:
# Write out the parcel IDs and addresses to upload for geocoding using:
# https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form
# Choose "Census2010_Current" as these tract IDs match the CEJST data
address_df = pd.read_excel(r"C:/gh/BPS/atlanta/2024-06-17_23-58-23--1PropertyAndTenantDataCombinedAt.xlsx")
address_df_filtered = address_df[['Address','City', 'State', 'Zip']]
print(address_df_filtered)
#address_df_filtered.to_csv(r"C:/gh/BPS/atlanta/atlanta_addresses_for_geocoding.csv",header=False)

                           Address            City State         Zip
0                    Springwood Dr      Carrollton    GA       30116
1                5160 NE Alcovy Rd       Covington    GA       30014
2                   341 Conley Cir         Atlanta    GA       30354
3               6545 Haddington Ln         Suwanee    GA  30024-5308
4                2045 Highway 34 E          Newnan    GA  30265-1327
...                            ...             ...   ...         ...
62838             6305 Crescent Dr        Norcross    GA       30071
62839                  5500 S Expy     Forest Park    GA  30297-2512
62840            445 Hurricane Trl          Dacula    GA       30019
62841  2100-2200 W Park Place Blvd  Stone Mountain    GA       30087
62842                 86 S Cobb Dr        Marietta    GA       30060

[62843 rows x 4 columns]


In [4]:
address_df_filtered = address_df_filtered.dropna(subset=['Address'])
# Print the number of rows in the DataFrame after cleaning
print("Number of rows after removing blank addresses:", len(address_df_filtered))

Number of rows after removing blank addresses: 62841


In [5]:
# Split DF into max 9,999 rows
df_dict = {n: address_df_filtered.iloc[n:n+9999, :] 
    for n in range(0, len(address_df_filtered), 9999)}

# export dataframes for the census geocoder
for key in df_dict.keys():
    print(key)
    df_dict[key].to_csv(f'PropertyDataCombined_{key}.csv', index_label='ID')


0
9999
19998
29997
39996
49995
59994


In [6]:

### read new csvs with tracts from the census geocoder
dfs = []
for key in df_dict.keys():
    print(key)
    df_geolocater = pd.read_csv(rf'C:\gh\BPS\atlanta\geocoded_addresses\GeocodeResults_{key}.csv',
        names=[
            'Serial Number',
            'Input Address',
            'TIGER Address Range Match Indicator',
            'TIGER Match Type',
            'TIGER Output Address',
            'LongitudeAndLatitude',
            'Tigerline ID',
            'Tigerline ID Side',
            'State',
            'County',
            'Tract',
            'Block'],
            dtype=str, index_col=False)
    
    # Count the number of rows where 'TIGER Match Type' is 'No Match'
    no_match_count = df_geolocater[df_geolocater['TIGER Address Range Match Indicator'] == 'No_Match'].shape[0] 
    dfs.append(df_geolocater)

df_tracts = pd.concat(dfs)

# Count the number of rows where 'TIGER Match Type' is 'No Match'
no_match_count_total = df_tracts[df_tracts['TIGER Address Range Match Indicator'] == 'No_Match'].shape[0]
match_count_total = df_tracts[df_tracts['TIGER Address Range Match Indicator'] == 'Match'].shape[0]

if not no_match_count_total == 0:
    no_match_percent = (no_match_count_total/ df_tracts.shape[0])*100
    print(f'% of "No Match" entries {no_match_percent}')

print(f'Number of "No Match" entries {no_match_count_total}')
print(f'Total entries {df_tracts.shape[0]}')
print(f'Number of "Matches" {match_count_total}')


0
9999
19998
29997
39996
49995
59994
% of "No Match" entries 10.767247963340122
Number of "No Match" entries 6767
Total entries 62848
Number of "Matches" 55022


In [7]:
# Combine the State, County, and Tract into a single field
def census_tract_id(row):

    state = row['State']
    county = row['County']
    tract = row['Tract']
    gisjoin = f'{state}{county}{tract}'
       

    return gisjoin

In [8]:
# clean up addresses and create census_tract_id column
#df_tracts.replace({',': ''}, regex=True, inplace=True) # remove extra comma in address
#df_tracts['full_address'] = df_tracts['Input Address'].str.lower() # convert to lowercase
#df_tracts = normalize_common_address_comps(df_tracts, 'full_address')
#df_tracts['State'] = df_tracts['State'].str.lstrip('0') # strip the 0 from the front of State values
df_tracts['census_tract_id'] = df_tracts.apply(lambda row: census_tract_id(row), axis=1) # create census id column
df_tracts.columns

Index(['Serial Number', 'Input Address', 'TIGER Address Range Match Indicator',
       'TIGER Match Type', 'TIGER Output Address', 'LongitudeAndLatitude',
       'Tigerline ID', 'Tigerline ID Side', 'State', 'County', 'Tract',
       'Block', 'census_tract_id'],
      dtype='object')

In [10]:
# filter to matched addresses
df_tracts_match = df_tracts.loc[df_tracts['TIGER Address Range Match Indicator'] == 'Match']
df_tracts_match.shape[0]


55022

In [11]:
# Rename 'Unnamed: 0' to 'Serial Number' in df_tracts_match_simple
address_df.rename(columns={'Unnamed: 0': 'Serial Number'}, inplace=True)

# Check if 'Identified as disadvantaged' column exists before dropping it
if 'Identified as disadvantaged' in address_df.columns:
    address_df.drop(columns=['Identified as disadvantaged'], inplace=True)
    print("Column 'Identified as disadvantaged' dropped.")
else:
    print("Column 'Identified as disadvantaged' does not exist.")

df_tracts_match['Serial Number'] = df_tracts_match['Serial Number'].astype(int)
# Check data types
print(address_df['Serial Number'].dtype)
print(df_tracts_match['Serial Number'].dtype)

# Check the updated DataFrame
print(address_df)

Column 'Identified as disadvantaged' dropped.
int64
int64
       Serial Number                      Address  \
0                  0                Springwood Dr   
1                  1            5160 NE Alcovy Rd   
2                  2               341 Conley Cir   
3                  3           6545 Haddington Ln   
4                  4            2045 Highway 34 E   
...              ...                          ...   
62838          62838             6305 Crescent Dr   
62839          62839                  5500 S Expy   
62840          62840            445 Hurricane Trl   
62841          62841  2100-2200 W Park Place Blvd   
62842          62842                 86 S Cobb Dr   

                           Property Name  PropertyType  Star Rating  \
0                                    NaN  Multi-Family          2.0   
1                                    NaN        Office          2.0   
2                                    NaN    Industrial          1.0   
3                    

C:\Users\tcharan\AppData\Local\Temp\1\ipykernel_31572\373648345.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracts_match['Serial Number'] = df_tracts_match['Serial Number'].astype(int)


In [12]:
# Merge DataFrames on 'Serial Number'
df_md_costar_tracts = pd.merge(address_df, df_tracts_match, on='Serial Number', how='inner')

In [13]:
# Load the EEEJ
cejst = pd.read_csv(r"C:\gh\BPS\atlanta\1.0-communities.csv", low_memory=False, dtype=object)


In [14]:
# Merge CEJST data on census tract ID
bef = len(df_md_costar_tracts)
print(f'Before joining CEJST on geographies, Nonres had {bef} entries')

df_md_costar_tracts_merged = pd.merge(df_md_costar_tracts, cejst, left_on='census_tract_id', right_on='Census tract 2010 ID', how='inner')

aft = len(df_md_costar_tracts_merged)
print(f'After joining CEJST on census geographies, Nonres had {aft} entries')
nTomatch = bef - aft
if nomatch > 0:
    print(f'This means there were {nomatch} census geographies not matched. If this number is high, make sure you choose "Census2010_current on the geocoding site.')


Before joining CEJST on geographies, Nonres had 55022 entries
After joining CEJST on census geographies, Nonres had 55022 entries


In [15]:
df_md_costar_tracts_merged[df_md_costar_tracts_merged['Census tract 2010 ID'].isna()]['census_tract_id'].value_counts()
df_md_costar_tracts_merged.to_csv(r"C:\gh\BPS\atlanta\DisadvantagedStatusOfBuildingsAtlanta.csv")

In [22]:
print(df_md_costar_tracts_merged.shape)   # Get the shape (number of rows and columns)


(55022, 2349)


Empty DataFrame
Columns: [Serial Number, Address, Property Name, PropertyType, Star Rating, Energy Star, LEED Certified, Building Class, Building Status, Rent/SF/Yr, Secondary Type, Leasing Company Name, Leasing Company Contact, City, State_x, Zip, Last Sale Price, Year Built, Year Renovated, Typical Floor Size, Property ID, $Price/Unit, Affordable Type, Amenities, Anchor Tenants, Average Weighted Rent, Building Operating Expenses, Building Park, Building Tax Expenses, Cap Rate, Ceiling Ht, Closest Transit Stop, Closest Transit Stop Dist (mi), Closest Transit Stop Walk Time (min), Construction Material, Direct Services, For Sale Price/Room, Fund Name, Gas, Heating, Latitude, Longitude, Number Of Elevators, Number Of Loading Docks, Number Of Stories, Operational Status, Ops Expense, Ops Expense Per SF, Owner Address, Owner City State Zip, Owner Contact, Owner Phone, Parent Company, Power, Rent Type, Rooms, Tax Year, Taxes Per SF, Taxes Total, Vacancy %, Zoning, Calculated Lease Term, % of Building0, Address0, Employees0, Industry0, Location Phone0, Occupancy Type0, Property Type0, Rent/SF/Yr0, SF Occupied0, Secondary Type0, Star Rating0, Store Type0, Tenant Name0, Website0, % of Building1, Address1, Employees1, Industry1, Location Phone1, Occupancy Type1, Property Type1, Rent/SF/Yr1, SF Occupied1, Secondary Type1, Star Rating1, Store Type1, Tenant Name1, Website1, % of Building2, Address2, Employees2, Industry2, Location Phone2, Occupancy Type2, Property Type2, Rent/SF/Yr2, SF Occupied2, Secondary Type2, ...]
Index: []

[0 rows x 2349 columns]